In [1]:
# 必要なライブラリをインポートします。
import os
import requests
import json

In [2]:
TOKEN = "67749947-a6d5-4032-8dc1-3f1b215094fd"
DATA_SET = "10402ffd-fb70-4625-9b46-63cec87cd5a9" ##ASNARO-2のデータセットIDをセット

#所定の場所のデータ検索をする関数
# Specify dataset and search. Using /datasets/{dataset_id}/data-search/ API.
def scene_search(dataset_id, intersects=None, query={}, sortby=None, paginate=None):
    url = 'https://www.tellusxdp.com/api/traveler/v1/datasets/{}/data-search/'.format(dataset_id)
    headers = {
        "Authorization": "Bearer " + TOKEN,
        'Content-Type': 'application/json'
    }
    payloads = {}
    if intersects is not None:
        payloads['intersects'] = intersects
    if query is not None:
        payloads['query'] = query
    if isinstance(sortby, list):
        payloads['sortby'] = sortby
    if paginate is not None:
        payloads['paginate'] = paginate
    
    r = requests.post(url, headers=headers, data=json.dumps(payloads))
    print(r.status_code)
    if r.status_code != 200:
        print(r.content)
        raise ValueError('status error({}).'.format(r.status_code))
        
    return r.json()



In [3]:
# 検索条件を指定してデータを検索する関数（AOIを指定）

def get_request():
    ret = scene_search(
        DATA_SET,
        intersects={
            'type':'Polygon','coordinates':[
                [
                    [141.84938200356842,42.75105653576509],
                    [141.83924706503996,42.71645449666897],
                    [141.89461385885272,42.69893933684679],
                    [141.96255548380265,42.72252158826697],
                    [141.95711264644478,42.761943222877846],
                    [141.89968132811697,42.76359672954325],
                    [141.84938200356842,42.75105653576509]
                ]
            ]
        },
    )
    return ret

print(get_request())



200
{'type': 'FeatureCollection', 'features': [{'dataset_id': '10402ffd-fb70-4625-9b46-63cec87cd5a9', 'geometry': {'coordinates': [[[141.85921515024788, 42.69645878203438], [141.98184752828192, 42.7119268040681], [141.9608764018023, 42.80151114290716], [141.83806053574853, 42.78604096679968], [141.85921515024788, 42.69645878203438]]], 'type': 'Polygon'}, 'id': '7d47301d-9533-4be2-a010-31a86912e865', 'type': 'Feature', 'properties': {'sar:frequency_band': 'X', 'processing:level': 'L2.1', 'sat:relative_orbit': 126, 'start_datetime': '2020-09-05T08:12:58+00:00', 'end_datetime': '2020-09-05T08:13:00+00:00', 'tellus:name': 'AS201462808498-200905', 'created': '2021-12-13T14:25:18.087579+00:00', 'tellus:can_ordered': True, 'tellus:sat_frame': 8498, 'view:off_nadir': 36.184845, 'asnaro2:beam': '1', 'sar:observation_direction': 'right', 'sar:polarizations': 'HH', 'sar:instrument_mode': 'SP2', 'tellus:published_datetime': '2021-12-17T01:28:38.867529+00:00', 'sat:orbit_state': 'ascending', 'sar:p

In [4]:
# 'id': '7d47301d-9533-4be2-a010-31a86912e865'
# 'id': '2c9ccde2-3a8c-4cd2-aac5-4f0c3b33a4a4'

# idが「7d47301d-9533-4be2-a010-31a86912e865」の2020-09-05に撮影されたデータと、
# id「2c9ccde2-3a8c-4cd2-aac5-4f0c3b33a4a4」で2018-09-08に撮影されたデータを使用する。

SCENE_ID1 = '7d47301d-9533-4be2-a010-31a86912e865'
SCENE_ID2 = '2c9ccde2-3a8c-4cd2-aac5-4f0c3b33a4a4'

In [5]:
# ファイルID等を取得する関数
def get_files_list(dataset_id, data_id):
    url = 'https://www.tellusxdp.com/api/traveler/v1/datasets/{}/data/{}/files/'.format(dataset_id, data_id)
    headers = {
        "Authorization": "Bearer " + TOKEN,
        'content-type': 'application/json'
    }
    
    r = requests.get(url, headers=headers)
    assert r.status_code == 200
    return r.json()

print(get_files_list(DATA_SET, SCENE_ID1))
print(get_files_list(DATA_SET, SCENE_ID2))


{'results': [{'size_bytes': 894255, 'mime_type': 'image/jpeg', 'name': 'BRO-AS201462808498-200905___-SP2R1.1__A_.jpg', 'id': 1, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 10178, 'mime_type': 'application/xml', 'name': 'MET-AS201462808498-200905___-SP2R1.1__A_.xml', 'id': 2, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 5038506136, 'mime_type': 'image/tiff', 'name': 'SOR-IMG-HH-AS201462808498-200905___-SP2R1.1__A_.tif', 'id': 3, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 658236500, 'mime_type': 'image/tiff', 'name': 'AS201462808498-200905_webcog.tif', 'id': 4, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}, {'size_bytes': 455386, 'mime_type': 'application/pdf', 'name': 'EULA.pdf', 'id': 5, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': 

In [6]:
# 上記関数で取得したファイルID
FILE_ID1 = '4'
FILE_ID2 = '3'


In [7]:
# ファイルIDで指定したファイルのみの情報を抽出
def get_file_info(dataset_id, data_id, file_id):
    url = 'https://www.tellusxdp.com/api/traveler/v1/datasets/{}/data/{}/files/{}/'.format(dataset_id, data_id, file_id)
    headers = {
        "Authorization": "Bearer " + TOKEN,
        'content-type': 'application/json'
    }
    
    r = requests.get(url, headers=headers)
    return r.json()

FILE_INFO1 = get_file_info(DATA_SET, SCENE_ID1, FILE_ID1)
FILE_INFO2 = get_file_info(DATA_SET, SCENE_ID2, FILE_ID2)

print(FILE_INFO1)
print(FILE_INFO2)


{'size_bytes': 658236500, 'mime_type': 'image/tiff', 'name': 'AS201462808498-200905_webcog.tif', 'id': 4, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}
{'size_bytes': 643175320, 'mime_type': 'image/tiff', 'name': 'AS200355208498-180908_webcog.tif', 'id': 3, 'status': 'uploaded', 'is_downloadable': True, 'require_archived_file_download': False}


In [11]:
# Create download URL
def get_download_url(dataset_id, data_id, file_id):
    print(dataset_id)
    print(data_id)
    print(file_id)
    url = 'https://www.tellusxdp.com/api/traveler/v1/datasets/{}/data/{}/files/{}/download-url/'.format(dataset_id, data_id, file_id)
    print(url)
    headers = {
        "Authorization": "Bearer " + TOKEN,
        'Content-Type': 'application/json',
    }
    
    r = requests.post(url, headers=headers)
    assert r.status_code == 200
    return json.loads(r.content)

DOWNLOAD_URL1 = get_download_url(DATA_SET, SCENE_ID1, FILE_ID1)
DOWNLOAD_URL2 = get_download_url(DATA_SET, SCENE_ID2, FILE_ID2)

print(DOWNLOAD_URL1)
print(DOWNLOAD_URL2)


10402ffd-fb70-4625-9b46-63cec87cd5a9
7d47301d-9533-4be2-a010-31a86912e865
4
https://www.tellusxdp.com/api/traveler/v1/datasets/10402ffd-fb70-4625-9b46-63cec87cd5a9/data/7d47301d-9533-4be2-a010-31a86912e865/files/4/download-url/


AssertionError: 

In [10]:
#取得した情報から必要な情報を取り出す 1つ目の画像
URL1 = DOWNLOAD_URL1['download_url']
FILE_NAME1 = FILE_INFO1['name']
SIZE_BYTES1 = FILE_INFO1['size_bytes']

print('File Name: ' + FILE_NAME1)
print('File Size; ' + str(SIZE_BYTES1))

#ファイルを作成して取得したファイル内容を書き出す。1つ目の画像
response = requests.get(URL1)
with open(FILE_NAME1, "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            f.flush()




NameError: name 'DOWNLOAD_URL1' is not defined